<a href="https://colab.research.google.com/github/JCaballerot/Deep_learning_program/blob/main/Topicos_avanzados/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"><font size="5">CIFAR10</font></h1>


<center>
  <img src="https://symjax.readthedocs.io/en/latest/_images/sphx_glr_plot_cifar10_001.svg" width="800" height="300">
</center>


Este laboratorio se enfoca en la construcción, entrenamiento y optimización de un modelo de red neuronal convolucional (CNN) para clasificar imágenes del conjunto de datos CIFAR10, que consiste en 60,000 imágenes a color de 32x32 distribuidas en 10 clases diferentes.

### 1. Instalación de dependencias


Primero, instalamos scikeras, que es una interfaz de Scikit-learn para Keras, permitiéndonos usar herramientas de Scikit-learn como GridSearchCV para optimizar nuestros modelos de Keras.

In [1]:
%%capture
!pip3 install scikeras

---

### 2. Importación de librerías necesarias

Aquí importamos todas las librerías necesarias para nuestro laboratorio, incluyendo TensorFlow, Keras para la construcción del modelo, y Scikit-learn para la optimización de hiperparámetros.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.regularizers import l1
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier


---

### 3. Definición del modelo


En esta sección, definimos la arquitectura de nuestra red neuronal convolucional (CNN), especificando capas convolucionales, capas de pooling, y capas densas para la clasificación. También configuramos dinámicamente el optimizador, la tasa de aprendizaje y la función de pérdida.

In [3]:


def create_model(learning_rate=0.01, optimizer='adam', loss='sparse_categorical_crossentropy', l1_reg=0.01):
    model = Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Dropout(0.25),  # Agregar Dropout después de la capa de MaxPooling
        Flatten(),
        Dense(64, activation='relu', kernel_regularizer=l1(l1_reg)),  # Aplicar regularización L1 aquí
        Dropout(0.5),  # Agregar Dropout antes de la capa final
        Dense(10, activation='softmax', kernel_regularizer=l1(l1_reg))  # Aplicar regularización L1 aquí
    ])

    optimizer_choices = {
        'adam': Adam(learning_rate=learning_rate),
        'sgd': keras.optimizers.SGD(learning_rate=learning_rate),
        'rmsprop': keras.optimizers.RMSprop(learning_rate=learning_rate)
    }

    model.compile(optimizer=optimizer_choices[optimizer],
                  loss=loss,
                  metrics=['accuracy'])
    return model


### 4. Preprocesamiento de datos


Cargamos y preprocesamos el conjunto de datos CIFAR10, normalizando las imágenes y convirtiendo las etiquetas a formato one-hot para su uso con la función de pérdida categorical_crossentropy.

In [4]:
from tensorflow.keras.utils import to_categorical

In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 11s 0us/step


In [11]:
x_train.shape

(50000, 32, 32, 3)

In [6]:
# Muestrear 2000 datos aleatoriamente de cada conjunto (entrenamiento y prueba)
indices_train = np.random.choice(range(x_train.shape[0]), size = 10000, replace=False)
indices_test = np.random.choice(range(x_test.shape[0]), size = 10000, replace=False)

x_train_sampled = x_train[indices_train]
y_train_sampled = y_train[indices_train]

x_test_sampled = x_test[indices_test]
y_test_sampled = y_test[indices_test]

In [7]:

# Normalizar los datos
x_train_sampled, x_test_sampled = x_train_sampled / 255.0, x_test_sampled / 255.0

# Para usar 'categorical_crossentropy', necesitamos etiquetas en formato one-hot
y_train_one_hot = to_categorical(y_train_sampled, 10)
y_test_one_hot = to_categorical(y_test_sampled, 10)

### 5. Configuración de Gridsearch


Establecemos una grilla de hiperparámetros para explorar diferentes configuraciones de tasa de aprendizaje, tamaño de lote, optimizadores y funciones de pérdida, y preparamos nuestro modelo para la optimización.

In [8]:
param_grid = {
    'model__learning_rate': [0.001, 0.01],
    'batch_size': [64, 128],
    'model__optimizer': ['adam', 'sgd', 'rmsprop'],  # optimizador
    'model__loss': ['sparse_categorical_crossentropy', 'categorical_crossentropy'],  # función de pérdida
    'model__l1_reg': [0.001, 0.01, 0.1]  # Nuevos valores de regularización L1 para probar
}


### 6. Ejecución de la búsqueda de hiperparámetros y resultados


Finalmente, ejecutamos la búsqueda de hiperparámetros utilizando GridSearchCV, entrenando el modelo con diferentes configuraciones y evaluando su rendimiento. Los mejores parámetros se muestran al final, junto con el rendimiento de todas las configuraciones probadas.

In [9]:

model = KerasClassifier(build_fn = create_model, epochs = 10, verbose=1)  # Ajusta epochs según necesidad
grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = 1, cv = 3, return_train_score = True, verbose = 2)


In [10]:
# Usa y_train_one_hot si eliges 'categorical_crossentropy', y_train para 'sparse_categorical_crossentropy'
grid_result = grid.fit(x_train_sampled, y_train_sampled)


Fitting 3 folds for each of 72 candidates, totalling 216 fits


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/10
4/4 [==============================] - 4s 115ms/step - loss: 8.7981 - accuracy: 0.1100
Epoch 2/10
4/4 [==============================] - 0s 80ms/step - loss: 7.6332 - accuracy: 0.1750
Epoch 3/10
4/4 [==============================] - 0s 68ms/step - loss: 6.6036 - accuracy: 0.2000
Epoch 4/10
4/4 [==============================] - 0s 97ms/step - loss: 5.6938 - accuracy: 0.1900
Epoch 5/10
4/4 [==============================] - 0s 75ms/step - loss: 4.9799 - accuracy: 0.1900
Epoch 6/10
4/4 [==============================] - 0s 69ms/step - loss: 4.3961 - accuracy: 0.1600
Epoch 7/10
4/4 [==============================] - 0s 70ms/step - loss: 3.9651 - accuracy: 0.2250
Epoch 8/10
4/4 [==============================] - 0s 72ms/step - loss: 3.7397 - accuracy: 0.1600
Epoch 9/10
4/4 [==============================] - 0s 76ms/step - loss: 3.5173 - accuracy: 0.2200
Epoch 10/10
4/4 [==============================] - 0s 11ms/step
[CV] END batch_size=64, model__l1_reg=0.001, model__learning_r

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


4/4 [==============================] - 2s 42ms/step - loss: 8.7510 - accuracy: 0.1150
Epoch 2/10
4/4 [==============================] - 0s 42ms/step - loss: 7.5356 - accuracy: 0.1700
Epoch 3/10
4/4 [==============================] - 0s 43ms/step - loss: 6.4414 - accuracy: 0.1450
Epoch 4/10
4/4 [==============================] - 0s 37ms/step - loss: 5.5572 - accuracy: 0.1350
Epoch 5/10
4/4 [==============================] - 0s 36ms/step - loss: 4.8180 - accuracy: 0.1550
Epoch 6/10
4/4 [==============================] - 0s 45ms/step - loss: 4.2582 - accuracy: 0.1450
Epoch 7/10
4/4 [==============================] - 0s 55ms/step - loss: 3.8722 - accuracy: 0.1500
Epoch 8/10
4/4 [==============================] - 0s 60ms/step - loss: 3.6335 - accuracy: 0.1250
Epoch 9/10
4/4 [==============================] - 0s 63ms/step - loss: 3.4605 - accuracy: 0.2100
Epoch 10/10
4/4 [==============================] - 0s 16ms/step
[CV] END batch_size=64, model__l1_reg=0.001, model__learning_rate=0.001, m

...

[CV] END batch_size=128, model__l1_reg=0.1, model__learning_rate=0.01, model__loss=categorical_crossentropy, model__optimizer=rmsprop; total time=   0.3s


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/10
[CV] END batch_size=128, model__l1_reg=0.1, model__learning_rate=0.01, model__loss=categorical_crossentropy, model__optimizer=rmsprop; total time=   0.3s


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
108 fits failed out of a total of 216.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
108 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 760, in fit
    self._fit(
  File "/usr/local/lib/python3.10/dist-packages/scikeras

Epoch 1/10
3/3 [==============================] - 1s 64ms/step - loss: 8.8422 - accuracy: 0.1200
Epoch 2/10
3/3 [==============================] - 0s 74ms/step - loss: 7.8800 - accuracy: 0.1133
Epoch 3/10
3/3 [==============================] - 0s 66ms/step - loss: 6.9979 - accuracy: 0.1733
Epoch 4/10
3/3 [==============================] - 0s 67ms/step - loss: 6.2323 - accuracy: 0.1367
Epoch 5/10
3/3 [==============================] - 0s 67ms/step - loss: 5.5170 - accuracy: 0.1933
Epoch 6/10
3/3 [==============================] - 0s 78ms/step - loss: 4.9308 - accuracy: 0.2100
Epoch 7/10
3/3 [==============================] - 0s 66ms/step - loss: 4.4041 - accuracy: 0.2200
Epoch 8/10
3/3 [==============================] - 0s 66ms/step - loss: 3.9887 - accuracy: 0.2533
Epoch 9/10
3/3 [==============================] - 0s 67ms/step - loss: 3.6774 - accuracy: 0.2767
Epoch 10/10
3/3 [==============================] - 0s 72ms/step - loss: 3.5069 - accuracy: 0.2333


Analizar Variance y Bias del modelo

In [12]:
# Asume que grid_result es el resultado de tu GridSearchCV

mean_train = grid_result.cv_results_['mean_train_score']
std_train = grid_result.cv_results_['std_train_score']

mean_test = grid_result.cv_results_['mean_test_score']
std_test = grid_result.cv_results_['std_test_score']

params = grid_result.cv_results_['params']

results_df = pd.DataFrame(params)
results_df['Train mean Accuracy'] = mean_train
results_df['Test mean Accuracy'] = std_train

results_df['Train std Accuracy'] = mean_train
results_df['Test std Accuracy'] = std_test


# Opcionalmente, podrías querer añadir una columna ID para identificar cada configuración
results_df['Config ID'] = results_df.index




In [14]:
results_df['variance'] = results_df['Train mean Accuracy'] + results_df['Test mean Accuracy']

In [16]:
results_df.sort_values(by = 'variance', ascending = True).head(30)

,batch_size,model__l1_reg,model__learning_rate,model__loss,model__optimizer,Train mean Accuracy,Test mean Accuracy,Train std Accuracy,Test std Accuracy,Config ID,variance
49,128,0.010,0.001,sparse_categorical_crossentropy,sgd,0.086667,0.010274,0.086667,0.028284,49,0.096941
13,64,0.010,0.001,sparse_categorical_crossentropy,sgd,0.108333,0.002357,0.108333,0.021602,13,0.110690
61,128,0.100,0.001,sparse_categorical_crossentropy,sgd,0.103333,0.016997,0.103333,0.036818,61,0.120330
26,64,0.100,0.001,sparse_categorical_crossentropy,rmsprop,0.113333,0.010274,0.113333,0.008165,26,0.123607
62,128,0.100,0.001,sparse_categorical_crossentropy,rmsprop,0.115000,0.014720,0.115000,0.004714,62,0.129720
37,128,0.001,0.001,sparse_categorical_crossentropy,sgd,0.116667,0.013123,0.116667,0.008165,37,0.129790
56,128,0.010,0.010,sparse_categorical_crossentropy,rmsprop,0.136667,0.008498,0.136667,0.012472,56,0.145165
18,64,0.010,0.010,sparse_categorical_crossentropy,adam,0.136667,0.008498,0.136667,0.012472,18,0.145165
32,64,0.100,0.010,sparse_categorical_crossentropy,rmsprop,0.143333,0.002357,0.143333,0.004714,32,0.145690
66,128,0.100,0.010,sparse_categorical_crossentropy,adam,0.143333,0.002357,0.143333,0.004714,66,0.145690


In [21]:
# Selecciona el hiperparámetro para visualizar
hyperparam = 'model__l1_reg'  # Este es el hiperparámetro de interés

# Preparando los datos para el gráfico
# Agrupando por el hiperparámetro seleccionado y calculando la media y desviación estándar de la precisión para cada valor
grouped = results_df.groupby(hyperparam)['Test mean Accuracy'].agg(['mean', 'std']).reset_index()

# Crear un rango de índices para las etiquetas de categorías
x_labels = range(len(grouped[hyperparam]))

# Graficar los resultados
plt.figure(figsize=(10, 6))
plt.bar(x_labels, grouped['mean'], yerr=grouped['std'], color='skyblue', capsize=5)
plt.title(f'Impacto de {hyperparam.capitalize()} en la Precisión Media')
plt.xlabel(hyperparam.capitalize())
plt.ylabel('Precisión Media')

# Asignar etiquetas de categorías al eje x
plt.xticks(x_labels, grouped[hyperparam], rotation=45)

plt.tight_layout()
plt.show()


### 6. Conclusión

En este laboratorio, aplicamos técnicas de optimización de hiperparámetros en un modelo de red neuronal convolucional (CNN) para clasificar imágenes del conjunto de datos CIFAR-10. A través del uso de GridSearchCV con KerasClassifier, exploramos combinaciones de hiperparámetros críticos, incluyendo la tasa de aprendizaje, el tamaño de lote, el optimizador y la función de pérdida, buscando la configuración óptima que maximizara la precisión de nuestro modelo.

Los resultados obtenidos demuestran que la optimización de hiperparámetros juega un papel crucial en el desempeño de los modelos de aprendizaje profundo. Identificamos la combinación de hiperparámetros que proporcionó la mayor precisión en la clasificación de las imágenes CIFAR-10, lo cual subraya la importancia de un proceso de selección y ajuste meticuloso.

Este laboratorio también destaca la potencia y flexibilidad de scikeras y GridSearchCV para integrar modelos de Keras en flujos de trabajo de aprendizaje automático con scikit-learn, facilitando así la experimentación y la optimización de modelos de aprendizaje profundo.

---
## Gracias por completar este laboratorio!